### Lab 6
- $i \in I=$ Nurse Names
- $j \in J=\{0,1,2,...\}$ Shift Index
- $n \in N=\{2,5,8,...\}$ Night Shift Index
- $w \in W=\{0,1,2,...\}$ Week Index

- Let your decision variables be $x_{ij}$, which is a binary variable for each nurse name $i$ for each shift $j$, $x_{ij} \in \{0,1\}$.
    - 0: Not assigned to the corresponding shift
    - 1: Assigned to the corresponding shift
- Let additional decision variables be $U_{all}$, $L_{all}$, $U_{night}$, and $L_{night}$, which are,
    - $U_{all}$: the maximum number of shifts worked by any nurse in this 9 week period
    - $L_{all}$: the minimum number of shifts worked by any nurse in this 9 week period
    - $U_{night}$: the maximum number of night shifts worked by any nurse in this 9 week period
    - $L_{night}$: the minimum number of night shifts worked by any nurse in this 9 week period
- For each nurse $i \in I$ for each shift $j \in J$, the preference score is $S_{ij}$, which is $S_{ij} \in \{0,1,2\}$.
    - 0: the shift is blacked-out, and he/she cannot be assigned to the shift under any circumstance
    - 1: the time is not preferred, but the nurse is willing to work if needed
    - 2: a preferred time slot
- For each shift $j \in J$, the shift requirement is $c_{j}$.
  
  
$$\begin{aligned}
\text{maximize} && \sum_{i \in I}\sum_{j \in J} S_{ij}x_{ij} - 100(U_{all} - L_{all}) - 100(U_{night} - L_{night})\\
\text{Maximum number of all shifts}&& \sum_{j \in J} x_{ij} \le U_{all} & \qquad \text{for all $i \in I$}&\\
\text{Minimum number of all shifts}&& \sum_{j \in J} x_{ij} \ge L_{all} & \qquad \text{for all $i \in I$}&\\
\text{Maximum number of all night shifts}&& \sum_{n \in N} x_{in} \le U_{night} & \qquad \text{for all $i \in I$}&\\
\text{Minimum number of all night shifts}&& \sum_{n \in N} x_{in} \ge L_{night} & \qquad \text{for all $i \in I$}&\\
\text{(Shift Requirement)} \\
\text{Blackout}&& x_{ij} \le S_{ij} & \qquad \text{for all $i \in I$, and $j \in J$}&\\
\text{Each shift}&& \sum_{i \in I} x_{ij} \ge c_{j} & \qquad \text{for all $j \in J$}&\\
\text{Each shift}&& \sum_{i \in I} x_{ij} \le c_{j} & \qquad \text{for all $j \in J$}&\\
\text{Each week}&& \sum_{w \in W} \sum_{k=21w}^{21w+21} x_{ik} \le 6 & \qquad \text{for all $w \in W$}&\\
\text{Prevention of consecutive shift}&& x_{ij} + x_{ij+1} \le 1 & \qquad \text{for all $j \in J$}\text{ except for last j}&\\\\
\text{(Night Shift Requirement)} \\
\text{Prevention of consecutive shift}&& x_{in} + x_{in+2} \le 1 & \qquad \text{for all $n \in N$}\text{ except for last n}&\\
\text{Prevention of consecutive shift}&& x_{in} + x_{in-2} \le 1 & \qquad \text{for all $n \in N$}&\\\\
\text{Non-negativity}&&x_{ij} \ge 0 & \qquad \text{for all $i \in I$, and $j \in J$}&\\
\end{aligned}$$

In [83]:
import gurobipy as grb
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math

file_name = 'small_data'
data = pd.ExcelFile('{}.xlsx'.format(file_name))

courses = pd.read_excel('{}.xlsx'.format(file_name), sheet_name='Course_Enrollment', parse_dates=False)#data.parse(0, parse_dates=True) #Courses
courses = courses.set_index(['Course', 'Section'])

rooms = data.parse(1) #Rooms
rooms.set_index('Room')

#students = data.parse(2) #Students

slots_per_day = 14 #AM8:00~PM10:00 total 14 hours, 28 time slots #Prime time AM10:00~PM5:00 14 slots
#num_days = 7

I = courses.index #Course name and section (tuple)
J = rooms.index #Room ID
K = ['M', 'T', 'W', 'H', 'F', 'S', 'U']
L = range(0,slots_per_day) #Time slot ID

D = courses.loc[:,'First Days']

s = courses.loc[:,'Reg Count'] #Registration Count
c = rooms.Size #Room Capacity

#Convert time difference to 30 mins unit slot: 3 h = 6 slots, 1 h 20 m = 3 slots
u2 = pd.to_datetime(courses.loc[:,'First End Time'].astype(str))#.apply(datetime)
u1 = pd.to_datetime(courses.loc[:,'First Begin Time'].astype(str))
u = ((u2-u1).apply(timedelta.total_seconds)/1800).apply(math.ceil).astype(int)

mod=grb.Model()

#Variable Definition
x={} #binary variable whether the course is assigned to each room at each time slot of each day
U={} #Maximum value of all slots for that day for each course and each room
for i in I:
    for j in J:
        for k in K:
            for l in L:
                x[i,j,k,l]=mod.addVar(vtype=grb.GRB.BINARY,name='x[{0},{1},{2},{3}]'.format(i,j,k,l))

#Total unit constraint : Included by below
            
#Day of week constraint
for i in I:
    for k in K:
        if k not in D[i]:
            mod.addConstr(sum(x[i,j,k,l] for j in J for l in L)==0)
        else:
            mod.addConstr(sum(x[i,j,k,l] for j in J for l in L)==u[i])

#Same slot of day constraint for multiple days
for i in I:
    if len(D[i]) > 1:
        for j in J:
            for index, d in enumerate(D[i]):
                if index < len(D[i])-1:
                    for l in L:
                        mod.addConstr(x[i,j,D[i][index],l]==x[i,j,D[i][index+1],l])
            
#Consecutive constraint
for i in I:
    for j in J:
        for d in D[i]:
            for index, l in enumerate(L):
                if index < len(L) - u[i] - 1:
                    mod.addConstr(x[i,j,d,l] + x[i,j,d,l+u[i]] <= 1)
                
#Room capacity constraint
for i in I:
    for j in J:
        for d in D[i]:
            for l in L:
                mod.addConstr(x[i,j,d,l]*s[i] <= 0.9*c[i])

#For each time slot of each room, one course constraint
for j in J:
    for k in K:
        for l in L:
            mod.addConstr(sum(x[i,j,k,l] for i in I) <= 1)

            
#Unique room constraint
for i in I:
    for j in J:
        for k in K:
            for l in L:
                mod.addConstr(x[i,j,k,l] <= U[i,j,k])
            mod.addConstr(sum(x[i,j,k,l] for l in L) == u[i]*U[i,j,k])

#Objective function


#Optimize
mod.optimize()
#print('Optimal Objective:', mod.ObjVal)

#Save output


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 33 entries, (DSO-621, 16321) to (DSO-599, 16318)
Data columns (total 24 columns):
Course Prefix            33 non-null object
Course Suffix            33 non-null int64
Department               33 non-null object
First Begin Time         33 non-null object
First Days               33 non-null object
First End Time           33 non-null object
First Instructor         33 non-null object
First Instructor UID     33 non-null int64
First Room               33 non-null object
Link                     0 non-null float64
Max Units                33 non-null float64
Min Units                33 non-null float64
Mode                     33 non-null object
Reg Count                33 non-null int64
Seats                    33 non-null int64
Second Begin Time        0 non-null float64
Second Days              0 non-null float64
Second End Time          0 non-null float64
Second Instructor        3 non-null object
Second Instructor UID    3 non-null

Course   Section
DSO-621  16321      6.0
DSO-522  16270      6.0
DSO-528  16274      6.0
DSO-401  16216      4.0
         16214      4.0
dtype: float64

In [48]:
pd.to_datetime('1:00:00 PM')

Timestamp('2018-04-07 13:00:00')